In [ ]:
# load necessay package
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import numpy as np
from keras.models import Model
import pandas as pd
import pickle
from sklearn import svm
from sklearn.model_selection import cross_val_score
import gc
from sklearn.multiclass import OneVsOneClassifier

### Functions to extract features from layers of VGG16 models

In [ ]:
def getfeatures(img_path,Models):
    features=[]
    #layername=['block1_pool','block2_pool','block3_pool','block4_pool','block5_pool']
    for i in range(5):
            model = Models[i]#Model(inputs=base_model.input, outputs=base_model.get_layer(layername[i]).output)
            img = image.load_img(img_path, target_size=(645, 484))
            x = image.img_to_array(img)
            x = np.expand_dims(x, axis=0)
            x = preprocess_input(x)
            block_pool_features = model.predict(x)
            block_pool_features=np.mean(block_pool_features,axis=(1,2))
            features.append(block_pool_features)
    Models=[]
    return features

def getfinalfeatures(img_path,model):
    img = image.load_img(img_path, target_size=(645, 484))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    features = model.predict(x)
    return features 

### Read image names and corresponding phases from excel file and split them based on phase names

In [ ]:
filename='micrograph.xlsx'
dataread=pd.read_excel(filename,'micrograph',index_col=None, na_values=['NA'])
head=dataread.columns.values.tolist()
constituents=dataread["primary_microconstituent"].values.tolist()
microid=dataread["micrograph_id"].values.tolist()
spheroiditename=[]
networkname=[]
perlitename=[]
Widmanstattenname=[]
pearandsphname=[]
pearandwidname=[]
martensitename=[]
for i in range(len(dataread)):
    if constituents[i]=='spheroidite':
        spheroiditename.append(microid[i])
    if constituents[i]=='network':
        networkname.append(microid[i])
    if constituents[i]=='pearlite':
        perlitename.append(microid[i])
    if constituents[i]=='spheroidite+widmanstatten':
        Widmanstattenname.append(microid[i])
    if constituents[i]=='pearlite+spheroidite':
        pearandsphname.append(microid[i])
    if constituents[i]=='pearlite+widmanstatten':
        pearandwidname.append(microid[i])
    if constituents[i]=='martensite':
        martensitename.append(microid[i])
spheroiditetrain=spheroiditename[0:100]
spheroiditetest=spheroiditename[100:]
networktrain=networkname[0:100]
networktest=networkname[100:]
perlitetrain=perlitename[0:100]
perlitetest=perlitename[100:]
Widmanstattentrain=Widmanstattenname[0:60]
Widmanstattentest=Widmanstattenname[60:]
trainset=[spheroiditetrain,networktrain,perlitetrain,Widmanstattentrain]
testset=[spheroiditetest,networktest,perlitetest,Widmanstattentest]
yy=['spheroidite','network','perlite','Widmanstatten']

### load VGG16 model with different intermediate layer outputs

In [ ]:
base_model = VGG16(weights='imagenet', include_top=False)
layername=['block1_pool','block2_pool','block3_pool','block4_pool','block5_pool']
Models=[]
for i in range(5):
    Models.append(Model(inputs=base_model.input, outputs=base_model.get_layer(layername[i]).output))

### Use different intermediate layers of VGG16 to extract features from images

In [ ]:
features=[0,0,0,0]
print('start features mid layers')
for ii in range(4):
    class1=trainset[ii]
    features[ii]=[]
    for i in range(len(class1)):
        img_path="micrograph"+str(class1[i])+"[cropped].tif"
        fresult=getfeatures(img_path,Models)
        features[ii].append(fresult)
        gc.collect()
        print('finish '+str(ii)+' '+str(i))
    print('finish '+str(ii))        
    with open('objs.pkl', 'wb') as f:  # Python 3: open(..., 'wb')
        pickle.dump(features, f)            
    f.close()            
with open('objs.pkl','rb') as f:  # Python 3: open(..., 'rb')
    features = pickle.load(f)
f.close()

### Evaluate the performance of the features extracted by different layers using SVM classifier, pick the most accurate feature-extracting layer

In [ ]:
classifierscore=[0,0,0,0,0,0]
classifiernames=[0,0,0,0,0,0]
nn=0   
print('start CV')
for ii in range(4-1):
    for jj in range(ii+1,4):
        X1=features[ii]
        y1=[yy[ii]]*len(X1)
        X2=features[jj]
        y2=[yy[jj]]*len(X2)
        y=y1[:]
        classifierscore[nn]=[]
        classifiernames[nn]=yy[ii]+" "+yy[jj]
        for n in y2:
            y.append(n)
        for i in range(5):
            X=[]
            for n in range(len(X1)):
                XX1=X1[n][i][0]
                X.append(XX1.flatten())
            for n in range(len(X2)):
                XX2=X2[n][i][0]
                X.append(XX2.flatten())
            clf = svm.SVC()
            this_scores = cross_val_score(clf, X, y, scoring='accuracy',cv=10)
            print('finish'+str(nn)+str(i))
            mean_score=this_scores.mean()
            classifierscore[nn].append(1-mean_score)
        print('finish '+str(nn))
        nn=nn+1

# write the results to an excel file. 
classifierdictionary={}
for i in range(6):
    classifierdictionary[classifiernames[i]]=classifierscore[i]
Results = pd.DataFrame(classifierdictionary, index=["C12","C22","C33","C43","C53"])
writer = pd.ExcelWriter('2_a.xlsx')
Results.to_excel(writer,'Sheet1')
writer.save()

### Extract features from image using "C53" layer (last layer before top layer) of VGG16 model and save it in file

In [ ]:
model = VGG16(weights='imagenet',include_top=False)
featuresfinal=[0,0,0,0]
for ii in range(4):
    class1=trainset[ii]
    featuresfinal[ii]=[]
    for i in range(len(class1)):
        img_path="micrograph"+str(class1[i])+"[cropped].tif"
        featuresfinal[ii].append(getfinalfeatures(img_path,model))
        gc.collect()
        print('done '+str(i))
print('done with features train\n')
featuresfinaltest=[0,0,0,0]
for ii in range(4):
    class1=testset[ii]
    featuresfinaltest[ii]=[]
    for i in range(len(class1)):
        img_path="micrograph"+str(class1[i])+"[cropped].tif"
        featuresfinaltest[ii].append(getfinalfeatures(img_path,model))
        gc.collect()
print('done with features test\n')
featuresfinalpands=[]
class1=pearandsphname
for i in range(len(class1)):
    img_path="micrograph"+str(class1[i])+"[cropped].tif"
    featuresfinalpands.append(getfinalfeatures(img_path,model))
    gc.collect()

print('done with features pands\n')
featuresfinalpandw=[]
class1=pearandwidname
for i in range(len(class1)):
    img_path="micrograph"+str(class1[i])+"[cropped].tif"
    featuresfinalpandw.append(getfinalfeatures(img_path,model))
    gc.collect()
print('done with features pandw\n')
featuresfinalmar=[]
class1=martensitename
for i in range(len(class1)):
    img_path="micrograph"+str(class1[i])+"[cropped].tif"
    featuresfinalmar.append(getfinalfeatures(img_path,model))
    gc.collect()
print('features done\n')
with open('objsfinal.pkl', 'wb') as f:  # Python 3: open(..., 'wb')
    pickle.dump([featuresfinal,featuresfinaltest,featuresfinalpands,featuresfinalpandw,featuresfinalmar], f)            
f.close()  

### Build one-vs-one binary classifiers using SVM

In [ ]:
with open('objsfinal.pkl','rb') as f:  # Python 3: open(..., 'rb')
    featuresfinal,featuresfinaltest,featuresfinalpands,featuresfinalpandw,featuresfinalmar = pickle.load(f)
f.close()

classifiernames=[0,0,0,0,0,0]
biclassifers=[]
nn=0

for ii in range(4-1):
    for jj in range(ii+1,4):
        X1=featuresfinal[ii]
        y1=[yy[ii]]*len(X1)
        X2=featuresfinal[jj]
        y2=[yy[jj]]*len(X2)
        y=y1[:]
        classifiernames[nn]=yy[ii]+" "+yy[jj]
        for n in y2:
            y.append(n)
        X=[]
        for n in range(len(X1)):
            XX1=np.mean(X1[n][0],axis=(0,1))
            #XX1=X1[n][4][0]
            X.append(XX1.flatten())
        for n in range(len(X2)):
            XX2=np.mean(X2[n][0],axis=(0,1))
            #XX2=X2[n][4][0]
            X.append(XX2.flatten())
        X=np.array(X)
        biclassifers.append(svm.SVC(C=1,class_weight="balanced",decision_function_shape="ovo",tol=0.001).fit(X,y))
        nn=nn+1

### Build multi-class classifier using voting

In [ ]:
def voting(biclassifers,X):
    y=[]
    for j in range(len(biclassifers)):
       y.append(biclassifers[j].predict(X))        
    y=np.array(y)
    axis=0
    unique, indices = np.unique(y, return_inverse=True)
    return unique[np.argmax(np.apply_along_axis(np.bincount, axis, indices.reshape(y.shape),
                                None, np.max(indices) + 1), axis=axis)]
Xv=[]
yv=[]
for ii in range(4):
     X1=featuresfinal[ii]
     yv.extend([yy[ii]]*len(X1))
     for n in range(len(X1)):
        XX1=np.mean(X1[n][0],axis=(0,1))
        #XX1=X1[n][4][0]
        Xv.append(XX1.flatten())
    
voteclassifier= OneVsOneClassifier(svm.SVC(C=1,class_weight="balanced",decision_function_shape="ovo",tol=0.001)).fit(Xv, yv)


### Evaluate multi-class classifier using test set

In [ ]:
scores=[0,0,0,0]
votingscores=[]
for ii in range(4):
    X1=featuresfinaltest[ii]
    y1=[yy[ii]]*len(X1)
    y=y1[:]
    X=[]
    scores[ii]=[]
    for n in range(len(X1)):
        XX1=np.mean(X1[n][0],axis=(0,1))
        #XX1=X1[n][4][0]
        X.append(XX1.flatten())
    X=np.array(X)
    for j in range(len(biclassifers)):
        scores[ii].append(1-biclassifers[j].score(X,y))
    #yvoting=voting(biclassifers,X)
    yvoting=voteclassifier.predict(X)
    votingscore=0
    for i in range(len(y)):
        if y[i]==yvoting[i]:
            votingscore+=1
    votingscore=votingscore/len(y)
    votingscores.append(1-votingscore)


### Make predictions and write results to excel files

In [ ]:
X1=featuresfinalpands
X=[]
for n in range(len(X1)):
    XX1=np.mean(X1[n][0],axis=(0,1))
    #XX1=X1[n][4][0]
    X.append(XX1.flatten())
X=np.array(X)
yvoting=voting(biclassifers,X)
ybinary=biclassifers[1].predict(X)
ypandsvote=yvoting 
ypandsbina=ybinary

X1=featuresfinalpandw
X=[]
for n in range(len(X1)):
    XX1=np.mean(X1[n][0],axis=(0,1))
    #XX1=X1[n][4][0]
    X.append(XX1.flatten())
X=np.array(X)
yvoting=voting(biclassifers,X)
ybinary=biclassifers[5].predict(X)
ypandwvote=yvoting 
ypandwbina=ybinary

X1=featuresfinalmar
X=[]
for n in range(len(X1)):
    XX1=np.mean(X1[n][0],axis=(0,1))
    #XX1=X1[n][4][0]
    X.append(XX1.flatten())
X=np.array(X)
yvoting=voting(biclassifers,X)
ymarvote=yvoting 

classifierdictionary={}
classifierdictionary['microid']=martensitename
classifierdictionary['predicted label']=ymarvote
Results = pd.DataFrame(classifierdictionary, index=None)
writer = pd.ExcelWriter('marvote.xlsx')
Results.to_excel(writer,'Sheet1')
writer.save()

classifierdictionary={}
classifierdictionary['microid']=pearandsphname
classifierdictionary['predicted label']=ypandsvote
Results = pd.DataFrame(classifierdictionary, index=None)
writer = pd.ExcelWriter('pandsvote.xlsx')
Results.to_excel(writer,'Sheet1')
writer.save()

classifierdictionary={}
classifierdictionary['microid']=pearandsphname
classifierdictionary['predicted label']=ypandsbina
Results = pd.DataFrame(classifierdictionary, index=None)
writer = pd.ExcelWriter('pandsbina.xlsx')
Results.to_excel(writer,'Sheet1')
writer.save()

classifierdictionary={}
classifierdictionary['microid']=pearandwidname
classifierdictionary['predicted label']=ypandwbina
Results = pd.DataFrame(classifierdictionary, index=None)
writer = pd.ExcelWriter('pandwbina.xlsx')
Results.to_excel(writer,'Sheet1')
writer.save()

classifierdictionary={}
classifierdictionary['microid']=pearandwidname
classifierdictionary['predicted label']=ypandwvote
Results = pd.DataFrame(classifierdictionary, index=None)
writer = pd.ExcelWriter('pandwvote.xlsx')
Results.to_excel(writer,'Sheet1')
writer.save()
